In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from numpy.linalg import inv,qr,solve,lstsq
from mediation_simulation import *
from linReg import *

In [3]:
import pandas as pd

In [4]:
var={'n':3,
     'p_otu':8, 
     'p_metabolite':4, 
     'mediations':1,
     'mediation_func':simple_mediation_direct
    }

In [5]:
truth_coef={
    'beta11':-2,
    'beta21':2,
    'beta32':-1.5
}

<hr>

# Simple mediation direct
<hr>

## 0. Simulation


In [6]:
np.random.seed(0)
output=simulate_simple_mediation(**var)

In [9]:
producer,target=output["OTU"][:-1,:],output["OTU"][-1,:]
metabolite=output["Metabolite"]

In [10]:
display(pd.DataFrame(data=output["OTU"],index=output["OtuAnn"]["Species"]),
pd.DataFrame(data=metabolite,index=output["MetAnn"]["Species"]))

,0,1,2
OTU1,0.548814,0.715189,0.602763
OTU2,0.544883,0.423655,0.645894
OTU3,0.437587,0.891773,0.963663
OTU4,0.383442,0.791725,0.528895
OTU5,0.568045,0.925597,0.071036
OTU6,0.087129,0.020218,0.832620
OTU7,0.944669,0.521848,0.414662
OTU8,-5.585027,-0.406539,-4.835802


,0,1,2
M1,0.778157,0.870012,0.978618
M2,0.799159,0.461479,0.780529
M3 Mediator,0.118274,0.639921,0.143353
M4 Mediator,2.753774,0.301532,3.099079


## step 1


### $$\text{Y}=\beta_{10}+\beta_{11}\text{X}+\epsilon$$

In [135]:
X=np.vstack([np.ones((1,producer.shape[1])),producer])
Y=target

coef, vcov,sigma,_=linmodEst(X,Y)
b10,b11=coef[:,0],coef[:,1:]

pd.DataFrame({
    "truth beta11":np.hstack([np.zeros(X.shape[0]-2),truth_coef["beta32"]*truth_coef["beta21"]]),
    "fitted beta11":b11.tolist()[0],
})

,truth beta11,fitted beta11
0,0.0,-0.034249
1,0.0,-2.547287
2,0.0,1.805463
3,0.0,2.250916
4,0.0,2.553489
5,0.0,-2.154310
6,-3.0,-4.917459


## Step 2

### $$\text{Me}=\beta_{20}+\beta_{21}X+\epsilon_2$$

In [136]:
Me=metabolite
coef, vcov,sigma,_=linmodEst(X,Me)

b20,b21=coef[:,0],coef[:,1:]
pd.DataFrame({
    "truth beta21":np.hstack([np.zeros(X.shape[0]-2),truth_coef["beta21"]]),
    "fitted beta21":b21.tolist()[0],
})

,truth beta21,fitted beta21
0,0.0,0.156284
1,0.0,0.173330
2,0.0,0.220686
3,0.0,0.124577
4,0.0,0.036616
5,0.0,0.187403
6,2.0,0.153214


## step 3

### $$Y=\beta_{30}+\beta_{31}X+\beta_{32}\text{Me}+\epsilon_3$$

In [174]:
X_Me=np.vstack([X,Me])
coef, vcov,sigma,_=linmodEst(X_Me,Y)

b30,b31,b32=coef[:,0],coef[:,1:X.shape[0]],coef[:,X.shape[0]:]

display(
    pd.DataFrame({
        "truth beta31":np.hstack([np.zeros(X.shape[0]-1)]),
        "fitted beta31":b31.tolist()[0]}),
    pd.DataFrame({
        "truth beta32":np.hstack([np.zeros(var["p_metabolite"]-var["mediations"]),truth_coef["beta32"]]),
        "fitted beta32":b32.tolist()[0]})
)

,truth beta31,fitted beta31
0,0.0,0.058133
1,0.0,-0.086354
2,0.0,0.620411
3,0.0,0.307557
4,0.0,-0.248023
5,0.0,0.516948
6,0.0,-0.825864


,truth beta32,fitted beta32
0,0.0,0.086927
1,0.0,-0.355654
2,0.0,0.329101
3,-1.5,-1.723042


In [152]:
# beta31 is expected to be smaller in absolute value than
# the original mediation effect beta11

np.abs(b31)<np.abs(b11)

matrix([[False,  True,  True,  True,  True,  True,  True]])

<hr>

# Simple mediation
<hr>

In [176]:
np.random.seed(0)
var["mediation_func"]=simple_mediation

output=simulate_simple_mediation(**var)

producer,target=output["OTU"][:-1,:],output["OTU"][-1,:]
metabolite=output["Metabolite"]


display(pd.DataFrame(data=output["OTU"],index=output["OtuAnn"]["Species"]),
pd.DataFrame(data=metabolite,index=output["MetAnn"]["Species"]))

,0,1,2
OTU1,0.548814,0.715189,0.602763
OTU2,0.544883,0.423655,0.645894
OTU3,0.437587,0.891773,0.963663
OTU4,0.383442,0.791725,0.528895
OTU5,0.568045,0.925597,0.071036
OTU6,0.087129,0.020218,0.832620
OTU7,0.944669,0.521848,0.414662
OTU8,-7.474365,-1.450236,-5.665125


,0,1,2
M1,0.778157,0.870012,0.978618
M2,0.799159,0.461479,0.780529
M3 Mediator,0.118274,0.639921,0.143353
M4 Mediator,2.753774,0.301532,3.099079


*step 1-2 are skipped

## step 3

### $$Y=\beta_{30}+\beta_{31}X+\beta_{32}\text{Me}+\epsilon_3$$

In [180]:
X_Me=np.vstack([X,Me])
coef, vcov,sigma,_=linmodEst(X_Me,Y)

b30,b31,b32=coef[:,0],coef[:,1:X.shape[0]],coef[:,X.shape[0]:]

display(
    pd.DataFrame({
        "truth beta31":np.hstack([np.zeros(X.shape[0]-2),truth_coef["beta11"]]),
        "fitted beta31":b31.tolist()[0]}),
    pd.DataFrame({
        "truth beta32":np.hstack([np.zeros(var["p_metabolite"]-var["mediations"]),truth_coef["beta32"]]),
        "fitted beta32":b32.tolist()[0]})
)

,truth beta31,fitted beta31
0,0.0,-0.031594
1,0.0,-0.127202
2,0.0,0.912026
3,0.0,0.317174
4,0.0,-0.778383
5,0.0,1.079783
6,-2.0,-1.436934


,truth beta32,fitted beta32
0,0.0,0.061853
1,0.0,-0.539015
2,0.0,0.301035
3,-1.5,-1.985328


In [181]:
# beta31 is expected to be smaller in absolute value than
# the original mediation effect beta11

np.abs(b31)<np.abs(b11)

matrix([[ True,  True,  True,  True,  True,  True,  True]])

<hr>
<hr>
<hr>